In [1]:
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# data preprocessing:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE 
from sklearn.preprocessing import StandardScaler
# models:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
# evaluation:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc

# 0. Loading the data

In [2]:
data = pd.read_csv("haberman.csv")
data.head()

,Age,Year_of_op,Nbr_of_nodes,Surv_status
0,30,64,1,1
1,30,62,3,1
2,30,65,0,1
3,31,59,2,1
4,31,65,4,1


# 1. Preprocessing the data

We preprocess the data same way we did in the project one which consisted the removement of outliers (too high values in Number of node column), balancing the data, normalizing the data, recoding the output variable and dividing the data into training and testing set.

**1.1. Removing the outliers**

In [3]:
a = data[data["Nbr_of_nodes"] > 30]
print("Observations with too high value in Number of node column:\n", a)

Observations with too high value in Number of node column:
      Age  Year_of_op  Nbr_of_nodes  Surv_status
62    43          58            52            2
174   54          67            46            1
215   59          62            35            2


In [4]:
# removing those three observations:
data = data.drop([62, 174, 215])

**1.2. Recoding the output variable (Survival status)**

In [5]:
data["Surv_status"] = data["Surv_status"].replace([1, 2], [0, 1])

**1.3. Dividing the data into training and testing sets**

In [6]:
x_train, x_test, y_train, y_test = train_test_split(data.drop(columns=["Surv_status"]), data["Surv_status"],
                                                   test_size=0.3, random_state=50, stratify=data["Surv_status"])
print("Train set size:", x_train.shape)
print("Test set size:", x_test.shape)

Train set size: (212, 3)
Test set size: (91, 3)


In [7]:
print("Train output values:\n" + str(y_train.value_counts()))
print("\nTest output values:\n" + str(y_test.value_counts()))

Train output values:
0    157
1     55
Name: Surv_status, dtype: int64

Test output values:
0    67
1    24
Name: Surv_status, dtype: int64


**1.4. Balancing the data (using SMOTE method)**

In [8]:
sm = SMOTE(random_state=5)
x_train, y_train = sm.fit_resample(x_train, y_train)
x_test, y_test = sm.fit_resample(x_test, y_test)
print("Train set output size:\n" + str(y_train.value_counts()))
print("\nTest set output size:\n" + str(y_test.value_counts()))

Train set output size:
0    157
1    157
Name: Surv_status, dtype: int64

Test set output size:
0    67
1    67
Name: Surv_status, dtype: int64


**1.5. Normalizing the data**

In [9]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

# 2. The logistic regression model from the project one (baseline)

In the project one we implemented four machine learning models which were; logistic regression, logistic ridge regression, logistic lasso regression and logistic elastic net regression. After evaluating and comparing the models we saw that the logistic regression model without the regularization was the best one for our data. Becuase of that we will now implement the logistic regression model same way we did it in the project one to set it as an baseline for this project's models. In the end of this project it is interesting to compare all the models including logistic regression model to saw which one is the best for our data.

We will build the logistic regression model by only using number of nodes variable as an explanatory variable beacause we saw that with the other two explanatory variables the model got worse evaluation metrics than the model with only number of nodes explanatory variable.

In [ ]:
# training:
x_train_mod = x_train[:,2].reshape([len(x_train), 1])
regressor = LogisticRegression()
regressor_trnd = regressor.fit(x_train_mod, y_train)

# predicting:
x_test_mod = x_test[:,2].reshape([len(x_test), 1])
y_pred = regressor_trnd.predict(x_test_mod)
y_pred_prob = regressor_trnd.predict_proba(x_test_mod)

# evaluating:
print("*** LOGISTIC REGRESSION EVALUATION: ***")
cm_n = confusion_matrix(y_test, y_pred) 
tn_lr, fp_lr, fn_lr, tp_lr = confusion_matrix(y_test, y_pred).ravel()
print("tn:", tn_lr, ",", "fp:", fp_lr, ",", "fn:", fn_lr, ",", "tp:", tp_lr)
# accuracy:
accuracy_lr = accuracy_score(y_test, y_pred)
print("Accuracy:", round(accuracy_lr, 2))
# sensitivity:
sensitivity_lr = recall_score(y_test, y_pred)
print("Sensitivity:", round(sensitivity_lr, 2))
# classification report:
names = ["surv. status >= 5 years", "surv. status < 5 years"]
print(classification_report(y_test, y_pred, target_names=names))

# computing the ROC curve and AUC
fpr_lr, tpr_lr, treshold_lr = roc_curve(y_test, y_pred_prob[:,1])
auc_lr = auc(fpr_lr, tpr_lr)
# plotting
plt.plot([0, 1], [0, 1], 'r--')
plt.plot(fpr_lr, tpr_lr, "b", label="AUC: "+str(round(auc_lr, 3)))
plt.legend(loc='lower right')
plt.xlim([0, 1]), plt.ylim([0, 1])
plt.title("ROC logistic regression")
plt.xlabel("False positive rate"), plt.ylabel("True positive rate")
plt.show()

# 3. Models

After implementing and analyzing the parametric models (project one) we implement five non-parametric models and analyze their performance. We are going to implement three supervised learning models and two unsupervised learning models, which are K-Nearest Neighbours, Boosted trees, Artificial neural network, K-Means and DBSCAN. K-Means and DBSCAN models are unsupervised learning models.

After implementing the models we will analyze and compare their performance and pick the best one.

# 3.1. K-NN

We start with K-nearest neighbors classifier which is supervised learning model. The way how the model works is that it classifies the new observation based on the k nearest data points. For example if the k is five and three of the closest data points belongs to class one, the model classifies the new data point belonging to class one. So when we start to implement the model we have to find the optimal value of k. For that we use cross-validation and to score the models with a different k values we use accuracy because it takes into account the both true positive and negative classifications. After finding the optimal k-value we calculate some model evaluation merits.

In [ ]:
# finding the optimal k
param_grid = {"n_neighbors": range(1, 30)}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, scoring="accuracy", cv=5)
grid.fit(x_train, y_train)
print("The optimal k:", grid.best_params_)
print("The best accuracy score:", grid.best_score_)

# plotting the scores
scores = np.array(grid.cv_results_["mean_test_score"])
n_of_k = np.array(param_grid["n_neighbors"])
plt.figure()
plt.plot(n_of_k, scores, "o-")
plt.xlabel("k"); plt.ylabel("Accuracy")
plt.show()

From figure we can see that models with the k one and two got equal and the best accuracy score. Next we try to do cross-validation with three folds to hopefully make our decision easier. 

In [ ]:
param_grid = {"n_neighbors": range(1, 30)}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, scoring="accuracy", cv=3)
grid.fit(x_train, y_train)
print("The optimal k:", grid.best_params_)
print("The best accuracy score:", grid.best_score_)

# plotting the scores
scores = np.array(grid.cv_results_["mean_test_score"])
n_of_k = np.array(param_grid["n_neighbors"])
plt.figure()
plt.plot(n_of_k, scores, "o-")
plt.xlabel("k"); plt.ylabel("Accuracy")
plt.show()

With three folds we got little bit different result. Because k=2 got the best score in both cross-validtions we pick it to our model.

In [ ]:
# implementig the model with k=2
knn_model = KNeighborsClassifier(n_neighbors=2).fit(x_train, y_train)
y_pred = knn_model.predict(x_test)
y_pred_prob = knn_model.predict_proba(x_test)

# evaluation metrics
print("*** K-NN EVALUATION: ***")
cm_knn = confusion_matrix(y_test, y_pred)
tn_knn, fp_knn, fn_knn, tp_knn = confusion_matrix(y_test, y_pred).ravel()
print("tn:", tn_knn, ",", "fp:", fp_knn, ",", "fn:", fn_knn, ",", "tp:", tp_knn)
# accuracy:
accuracy_knn = accuracy_score(y_test, y_pred)
print("Accuracy:", round(accuracy_knn, 2))
# sensitivity:
sensitivity_knn = recall_score(y_test, y_pred)
print("Sensitivity:", round(sensitivity_knn, 2))
# classification report:
names = ["surv. status >= 5 years", "surv. status < 5 years"]
print(classification_report(y_test, y_pred, target_names=names))

# ROC curve and AUC
fpr_knn, tpr_knn, treshold_knn = roc_curve(y_test, y_pred_prob[:,1])
auc_knn = auc(fpr_knn, tpr_knn)
# plotting
plt.plot([0, 1], [0, 1], 'r--')
plt.plot(fpr_knn, tpr_knn, "b", label="AUC: "+str(round(auc_knn, 3)))
plt.legend(loc='lower right')
plt.xlim([0, 1]), plt.ylim([0, 1])
plt.title("ROC of K-NN")
plt.xlabel("False positive rate"), plt.ylabel("True positive rate")
plt.show() 

**THE K-NN MODEL EVALUATION:**
- **The accuracy = 0.55:** Shows that the model classified only little bit over half of the test set patients correctly.
- **The precision:** Shows that the model classified correctly 53% of the test set's patients which it classified to belong to the class survival status five or more years and 65% of the patients which it classified to belong to the survival status less than five years class.
- **The recall:** Shows that the model classified the patients who belong to the class survival status five or more years 88% correctly and the most of the patients who belong to the class survival status less than five years wrong (only 22% correctly).
- **F1-score:** Because the f1-score is the average of the precision and recall it allows us to compare which class the model classified better. Because the class with the survival status five or more years has higher score than less than five years class the model classified it better.

**As an conclusion we can say that model did not perform well to classify the test set because the evaluation scores are pretty low. Also the AUC value is not so good.**

# 3.2. Gradient Boosting Trees

Next we implement the Gradient boosting trees model which consists many simple decision trees but the difference to the Random forest models is that in Gradient boosting trees every new decision tree in model tries to correct the error the previous ones has done. The way this method practically works is that when the model starts to train new tree to the model the observations previous trees classified wrong has a weight over one so this new tree has bigger probability to classify those obervations right. When all the decision trees have been trained the classification of the new data is a linear combination of the individual decision trees classifications.

When starting to implement the Gradient boosting trees model we have to optimize three parameters which are number of estimators, learning rate and maximum depth. Number of estimators means the amount of decision trees in the model. Learning rate is a parameter which tries to prevent model not to overfit. When setting learning rate small the model "can't learn so well". The maximum depth parameter defines how many variables the decision trees consists. Because the idea in the Gradient boosting trees is that we combine many bad performing trees, we wan't to set the depth of trees pretty small.

In [ ]:
# finding the optimal parameters:
model = GradientBoostingClassifier(random_state=5)
param_grid = {"n_estimators" : [50, 100, 200, 500, 1000],
             "learning_rate" : [0.1, 0.05, 0.01, 0.005, 0.001],
             "max_depth" : [1, 2, 3]}
grid = GridSearchCV(model, param_grid, scoring="accuracy", cv=5)
grid.fit(x_train, y_train)
print("Optimal parameters:", grid.best_params_)
print("The best accuracy score:", grid.best_score_)

In [ ]:
# building the model using the optimal parameters:
model = GradientBoostingClassifier(learning_rate=0.1, max_depth=3, n_estimators=500, random_state=5).fit(x_train, y_train)
# prediction:
y_pred = model.predict(x_test)
y_pred_prob = model.predict_proba(x_test)

# evaluation metrics
print("*** GRADIENT BOOSTING TREES EVALUATION: ***")
cm_gbt = confusion_matrix(y_test, y_pred)
tn_gbt, fp_gbt, fn_gbt, tp_gbt = confusion_matrix(y_test, y_pred).ravel()
print("tn:", tn_gbt, ",", "fp:", fp_gbt, ",", "fn:", fn_gbt, ",", "tp:", tp_gbt)
# accuracy:
accuracy_gbt = accuracy_score(y_test, y_pred)
print("Accuracy:", round(accuracy_gbt, 2))
# sensitivity:
sensitivity_gbt = recall_score(y_test, y_pred)
print("Sensitivity:", round(sensitivity_gbt, 2))
# classification report:
names = ["surv. status >= 5 years", "surv. status < 5 years"]
print(classification_report(y_test, y_pred, target_names=names))

# ROC curve and AUC
fpr_gbt, tpr_gbt, treshold_gbt = roc_curve(y_test, y_pred_prob[:,1])
auc_gbt = auc(fpr_gbt, tpr_gbt)
# plotting
plt.plot([0, 1], [0, 1], 'r--')
plt.plot(fpr_gbt, tpr_gbt, "b", label="AUC: "+str(round(auc_gbt, 3)))
plt.legend(loc='lower right')
plt.xlim([0, 1]), plt.ylim([0, 1])
plt.title("ROC of GBT")
plt.xlabel("False positive rate"), plt.ylabel("True positive rate")
plt.show() 

**THE GRADIENT BOOSTING TREES MODEL EVALUATION:**
- **The accuracy = 0.57:** Shows that the model classified almost 60% of the test dataset correctly.
- **The precision:** Tells us that the model classified 56% of the patients correctly which it classified to belong to the survival status five or more years class and 60% of the patients correctly which it classified to belong to the survival status less than five years class.
- **The recall:** Shows us that from all the patients who belong to the survival status five or more years class the model classified correctly 69% and from all the patients who belong to the survival status less than five years class the model classified correctly only 46%.
- **The F1-score:** Shows that the model classified better the class survival status five or more years but actually not so good either class.

**As a conclusion we can say that the Gradient boosting trees model didn't perform well in classifying the test dataset because the accuracy, precision, recall, AUC and f1-scores are pretty low.**

**----------**

**Next** we examine the importance of the variables in this model by examining the importance score of the input variables. Basically the importance score means that the higher the score is the more the variable is used in the model's trees to classify the patients.

In [ ]:
importances = pd.Series(model.feature_importances_, ["Age", "Year of op.", "Number of nodes"])
plt.figure()
importances.plot.bar()
plt.ylabel("Variable importance")
plt.show()

From the figure we can see that the Age variable seems to be the most important variable in this model which is totally different result than we get in our baseline model. In our baseline model (logistic regression) we saw that only Number of nodes variable is statistically significant which means that Age and Year of operation variables are insignificant when classifiyng the patients. When we compare the evaluation metrics between this model and baseline model we can see that the baseline model classified better the test set patients than this model so it seems that maybe Number of node variable is actually more important when classifiyng the patients.

# 3.3. Multilayer Perceptron

Next we implement our last supervised learning model which is artificial neural network model Multilayer perceptron. Artificial neural network models tries to copy biological neural networks. The artificial neural networks consist of input layer, hidden layer/layers (can have differet amount of hidden layers) and output layer which consist of nodes. The activation functions are used to calculate if the node will activate or not based on the previous layer's nodes. 

Artificial neural network models has a lot of factors to be determined for example activation functions between different layers, the amount of hidden layers, learning rate etc. Because of that we are not going to search the optimal option for all of the factors. What we are going to do is that we use the default options in most of the factors and we will optimize the amount of hidden layers and nodes in hidden layers and the weight of the regularization term (= alpha). As an activation function between the hidden layers we will use the rectified linear unit function.

In [15]:
import warnings
warnings.filterwarnings('ignore')

# finding the optimal parameters
param_grid = {"alpha" : [1e-2, 1, 5, 10, 20, 30], 
             "hidden_layer_sizes" : [(5,), (10,), (5,5), (10,10), (5,10), (10,5), (15,15)]} # (amount of nodes, amount of layers)
grid = GridSearchCV(MLPClassifier(random_state=3), param_grid, scoring="accuracy", cv=5)
grid.fit(x_train, y_train)
print("The optimal parameters:", grid.best_params_)
print("The accuracy score:", grid.best_score_)

The optimal parameters: {'alpha': 0.01, 'hidden_layer_sizes': (10, 10)}
The accuracy score: 0.6528929851510495


In [17]:
# implementing the model using optimal parameters
model = MLPClassifier(random_state=3, alpha=0.01, hidden_layer_sizes=(10,10)).fit(x_train, y_train)
y_pred = model.predict(x_test)
y_pred_prob = model.predict_proba(x_test)

# evaluation metrics
